In [1]:
import pandas as pd 
import process
import numpy as np 
# Jerome path : r'C:\Users\33640\OneDrive\Documents\GitHub\Portfolio_clustering_project\Data\DataBase.csv'
# Nail path : '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DataBase.csv'
df = pd.read_csv(r'C:\Users\33640\OneDrive\Documents\GitHub\Portfolio_clustering_project\Data\DataBase.csv')

df.set_index('ticker', inplace=True)

df.columns = pd.to_datetime(df.columns.str[1:], format='%Y%m%d').strftime('%d/%m/%Y')

df_cleaned = df.fillna(0) # Utilisez la méthode fillna(0) pour remplacer les NaN par 0

df_cleaned = df_cleaned.transpose() ## WE WANT COLUMNS TO BE VECTOR OF RETURN FOR A GIVEN TICKER

c:\Users\33640\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\33640\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
C:\Users\33640\AppData\Local\Temp/ipykernel_33544/396982650.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [21]:
beta = 0.95
K = 4  # Number of fold for the cross validation
N = 695  # Number of assets
Ik_length = 50  # Number of days in each fold for the cross validation

# Initialize epsilon as a zero array with N elements
epsilon = np.zeros(N)


for k in range(K):
    # Calculate EWA matrix 
    weighted_matrices = [(beta**(Ik_length-t)) * np.outer(df_cleaned.iloc[t + Ik_length*k], df_cleaned.iloc[t + Ik_length*k]) for t in range(Ik_length)]
    summed_weighted_matrices = np.sum(weighted_matrices, axis=0)
    E_matrix = (1 - beta) / (1 - beta**Ik_length) * summed_weighted_matrices
    
    # Calculate eigenvectors for the E matrix
    eigenvalues, eigenvectors = np.linalg.eigh(E_matrix)

    # Calculate epsilon terms for each eigenvector
    for i in range(N):
        ui = eigenvectors[:, i]
        # For each day in the Ik segment, project the data onto the eigenvector and square it
        epsilon_i_sum = np.sum([(np.dot(ui, df_cleaned.iloc[t + Ik_length*k])**2) for t in range(Ik_length)])
        # Accumulate the results in epsilon
        epsilon[i] += epsilon_i_sum.real / Ik_length

# Average epsilon over K segments
epsilon /= K

# Now, we calculate the forecasts using the last set of eigenvectors
Forecasts = np.sum([epsilon[i] * np.outer(eigenvectors[:, i], eigenvectors[:, i]) for i in range(N)], axis=0)

In [2]:
from PyFolioC import PyFolio
from PyFolioC import PyFolioC

historical_data = df_cleaned
number_of_repetitions = 10
lookback_window = [0, 50]
evaluation_window = 1
number_of_clusters = 38
clustering_method = 'SPONGE'
sigma = 0.1
eta = 0.1

consolidated_W = PyFolioC(number_of_repetitions=number_of_repetitions, historical_data=df_cleaned, lookback_window=lookback_window, evaluation_window=evaluation_window, number_of_clusters=number_of_clusters, sigma=sigma, eta=eta, clustering_method=clustering_method)

In [3]:
consolidated_W.sliding_window(10)

1.0044931623282323
step 1
1.0166687281847513
step 2
1.0195083623502044
step 3
1.0203762217597587
step 4
1.0256214503165464
step 5
1.0278326613983257
step 6
1.030928249800027
step 7
1.0304429207078931
step 8
1.0314772312816862
step 9
1.0320651616089225
step 10


(              return
 15/03/2000  0.004493
 16/03/2000  0.012121
 17/03/2000  0.002793
 20/03/2000  0.000851
 21/03/2000  0.005140
 22/03/2000  0.002156
 23/03/2000  0.003012
 24/03/2000 -0.000471
 27/03/2000  0.001004
 28/03/2000  0.000570,
 array([0.  , 0.02, 0.02, 0.02, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03]),
 [1,
  1.0044931623282323,
  1.0166687281847513,
  1.0195083623502044,
  1.0203762217597587,
  1.0256214503165464,
  1.0278326613983257,
  1.030928249800027,
  1.0304429207078931,
  1.0314772312816862,
  1.0320651616089225],
 array([ 0.  ,  0.01,  0.  ,  0.  ,  0.01,  0.  ,  0.  , -0.  ,  0.  ,
         0.  ]))

In [ ]:
def calcul_somme(beta):

    T = len(consolidated_W.historical_data.columns)
    
    res = np.zeros((len(df_column[0]), len(df_column[0])))

    res = (1 - beta) / (1 - beta**T) * np.sum([(beta**(T-(t+1))) * np.outer(consolidated_W.historical_data[ticker], consolidated_W.historical_data[ticker]) for t, ticker in enumerate(consolidated_W.historical_data.columns)])

    return res

beta = 0.5

res = calcul_somme(beta=beta)
res 
